**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
import IPython
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function $act$ proposes the next action to do from a certain state. 

```epsilon``` represents how likely the agent is going to explore actions randomly. More specifically, the agent will choose random action with a probability of $epsilon$. And will choose the action proposed by certain policies with a probability of $1-epsilon$. 
The higher the epsilon is, the more likely the agent is going to explore the rest of the graph randomly. 



***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [27]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0
        self.scale=16
        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, 
                                 grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, 
                        interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1 
        self.position[:,0:2] = -1 
        self.position[-2:, :] = -1 
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [25]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=100 # set small when debugging
epochs_test=20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

```position``` array is used for holding the ```state``` 'location features' of a certain state. More specifically, the locations which belong the marginal 2 grids, will be assigned $-1$ , the cell the rat stands at will be assigned $1$ and all other cells will remain $0$. And when the  rat makes an action, the position array will be reshaped and passed to the first element of variable 'state' to return in the end. 

```board``` array is used for holding the reward of a certain location. And if the location has been explored, the reward will changed to 0 afterwards. The state array will be reshaped and passed to the second element of variable 'state' to return in the end.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [1]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        action = np.random.randint(0,4)
        return action

NameError: name 'Agent' is not defined

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        state = env.reset()
        game_over = False
        
        win = 0
        lose = 0
        
        while not game_over:
            action = agent.act(state)
            prev_state = state
            state, reward, game_over = env.act(action)
             
            # update the counter 
            if reward > 0: 
                win += reward 
            if reward < 0:
                lose -= reward
            # Apply reinforcement strategy
            loss = agent.reinforce(prev_state,state,action,reward,game_over)
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score +=  win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 1.5/1.0. Average score (0.5)
Win/lose count 0.5/1.0. Average score (0.0)
Win/lose count 0.5/2.0. Average score (-0.5)
Win/lose count 2.0/1.0. Average score (-0.125)
Win/lose count 1.0/3.0. Average score (-0.5)
Win/lose count 2.0/1.0. Average score (-0.25)
Win/lose count 1.5/2.0. Average score (-0.2857142857142857)
Win/lose count 4.0/3.0. Average score (-0.125)
Win/lose count 3.5/2.0. Average score (0.05555555555555555)
Win/lose count 2.5/1.0. Average score (0.2)
Final score: 0.2


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




Q1: 
Because of Bellman Equation, we know that: 
\begin{equation*}
Q^{\pi}(s,a)=E_{\pi}[\Sigma^{\infty}_{t=0}\gamma^{t}(s_{t},\pi(s_{t}),s_{t+1})|s_{t}=s,a_{t}=a]
\end{equation*}

\begin{equation*}
Q^{\pi}(s_{0},a_{0})=E_{\pi}[r_{0}(s_{0},a_{0},s_{1}) + \Sigma^{\infty}_{t=1}\gamma^{t}(s_{t},\pi(s_{t}),s_{t+1})|s_{0},a_{0}]\\=E_{\pi}[r_{0}(s_{0},a_{0},s_{1}) + \gamma E_{\pi}[ \Sigma^{\infty}_{t=1}\gamma^{t-1}r_{t}(s_{t},\pi(s_{t}),s_{t+1})|s_{1},a_{0} | s_{0},a_{0}]\\= E_{\pi}[r_{0}(s_{0},a_{0},s_{1})+\gamma Q^{\pi}(s_{1},a_{1})]
\end{equation*}

Thus, \begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]\\
\\
\end{equation*}


Q2:
Because of Bellman Equation, we know that: 
\begin{equation*}
V^{\pi}(S_{0})=E[\Sigma^{\infty}_{t=0}\gamma^{t}(s_{t},\pi(s_{t}),s_{t+1})|s_{0}]
\end{equation*}
and 
\begin{equation*}
V^{*}(S_{0})=\max E[\Sigma^{\infty}_{t=0}\gamma^{t}r_{t}(s_{t},\pi(s_{t}),s_{t+1})|s_{0}]
\end{equation*}


Then:
\begin{equation*}
V^{*}(S_{0})=\max_{a'} E[r_{0}(s_{0},a{0},s_{1}) + \Sigma^{\infty}_{t=1}\gamma^{t}r_{t}(s_{t},\pi(s_{t}),s_{t+1})|s_{0}]\\=\max E_{s_{1}}[r_{0}(s_{0},a_{0},s_{1})+\gamma V^{*}(s_{1})]\\ V^{*}(s) = \max E_{s'}[r(s,a,s') + \gamma V^{*}(s')|s]
\end{equation*}

Since: 
\begin{equation*}
V^{*}(S_{0})=\max Q^{*}(s,a') \\
\end{equation*}

\begin{equation*}
V^{*}(s) = \max E_{s'}[r(s,a,s') + \gamma \max_{a'} Q^{*}(s',a')|s]\\
\end{equation*}
\begin{equation*}
Q^{*}(s,a) = E_{s'}[r(s,a,s') + \gamma \max_{a'} Q^{*}(s',a')|s,a]
\end{equation*}
Finally, we get: 
\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

Q3: 
The loss function is: 
\begin{equation*}
L(\theta) = [Q^{*}(s,a,\theta) - Q(s,a,\theta)]^{2}
\end{equation*}

Replace Q*(s,a,$\theta$) 
\begin{equation*}
L(\theta) = E_{s,a} [(E_{s'}[r(s,a,s') + \gamma\max Q(s',a,\theta)] - Q(s,a,\theta)]]^{2}
\end{equation*}

So that: 
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
import random 
import collections

class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        # initialize the memory to be zeros 
        self.memory = collections.deque(maxlen = self.max_memory)
    
    def remember(self, m):
         self.memory.append(m)

    def random_access(self):
        #Random choose actions
        return random.choice(self.memory)

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [16]:
from keras.layers import Flatten
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99   
        self.grid_size = grid_size
        self.epsilon = epsilon
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        #choose the one which max q 
        action = np.argmax(self.model.predict(s[np.newaxis,:]))
        return action
    
    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
            
        for i in range(self.batch_size):
            ######## FILL IN
            random_choose = self.memory.random_access()
            state = random_choose[0]
            next_state = random_choose[1]
            action = random_choose[2]
            reward = random_choose[3]
            game_over_ = random_choose[4]
            
            if game_over_:
                ######## FILL IN
                target_q[i][action] = reward 
                
            else:
                ######## FILL IN
                input_states[i] = state
                target_q[i][action] = reward + self.discount* np.max(self.model.predict(next_state[np.newaxis,:]))
                
               

        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args,lr = 0.1, **kwargs):
        super(DQN_FC, self).__init__( *args, **kwargs)
        
        # NN Model
        
        model = Sequential()
        model.add(Dense(128,input_shape = (5,5,self.n_state),activation ='relu' ))
        model.add(Flatten())
        model.add(Dense(4,activation='linear'))
        model.compile(sgd(lr=lr,decay =1e-4, momentum = 0.0 ),'mse')

        ####### FILL IN
        self.model = model 
        print(self.model.summary())

In [18]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=0.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 5, 5, 128)         384       
_________________________________________________________________
flatten_4 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 4)                 12804     
Total params: 13,188
Trainable params: 13,188
Non-trainable params: 0
_________________________________________________________________
None
Epoch 000/100 | Loss 0.0164 | Win/lose count 3.0/6.0 (-3.0)
Epoch 001/100 | Loss 0.0102 | Win/lose count 7.5/5.0 (2.5)
Epoch 002/100 | Loss 0.0031 | Win/lose count 5.5/5.0 (0.5)
Epoch 003/100 | Loss 0.0012 | Win/lose count 3.0/4.0 (-1.0)
Epoch 004/100 | Loss 0.0035 | Win/lose count 6.0/8.0 (-2.0)
Epoch 005/100 | Loss 0.0091 | Win/lose count 5.5/3.0 (2.5)
Epoch 006/100 | Loss 0.0122 | Win/lose c

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [21]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        
        # input convolution layer
        model.add(Conv2D(
            nb_filter = 32,
            nb_row = 5,
            nb_col = 5,
            border_mode = 'same', #padding method
            input_shape= (5,5,self.n_state),
            activation = 'relu',))
        
        model.add(MaxPooling2D(
        pool_size =(2,2),
        strides = (2,2),
        border_mode = 'same',))

        model.add(Conv2D(
            nb_filter = 64,
            nb_row = 5,
            nb_col = 5,
            border_mode = 'same', #padding method
            activation = 'relu',))
        
        model.add(MaxPooling2D(
        pool_size =(2,2),
        border_mode = 'same',))
        
        #Flatten
        model.add(Flatten())
        #ouput layer 
        model.add(Dense(4,activation='linear'))
    
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        print(self.model.summary())

In [24]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

/Users/mac/Downloads/Python/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(input_shape=(5, 5, 2), activation="relu", kernel_size=(5, 5), filters=32, padding="same")`
  from ipykernel import kernelapp as app
/Users/mac/Downloads/Python/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same")`
/Users/mac/Downloads/Python/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(activation="relu", kernel_size=(5, 5), filters=64, padding="same")`
/Users/mac/Downloads/Python/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:31: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), padding="same")`


Epoch 000/100 | Loss 0.0009 | Win/lose count 3.5/0 (3.5)
Epoch 001/100 | Loss 0.0081 | Win/lose count 0.5/6.0 (-5.5)
Epoch 002/100 | Loss 0.0089 | Win/lose count 3.0/7.0 (-4.0)
Epoch 003/100 | Loss 0.0003 | Win/lose count 2.5/2.0 (0.5)
Epoch 004/100 | Loss 0.0042 | Win/lose count 4.0/4.0 (0.0)
Epoch 005/100 | Loss 0.0004 | Win/lose count 4.0/4.0 (0.0)
Epoch 006/100 | Loss 0.0007 | Win/lose count 3.0/2.0 (1.0)
Epoch 007/100 | Loss 0.0019 | Win/lose count 2.5/0 (2.5)
Epoch 008/100 | Loss 0.0167 | Win/lose count 6.0/3.0 (3.0)
Epoch 009/100 | Loss 0.0082 | Win/lose count 4.5/2.0 (2.5)
Epoch 010/100 | Loss 0.0099 | Win/lose count 3.0/1.0 (2.0)
Epoch 011/100 | Loss 0.0019 | Win/lose count 4.5/1.0 (3.5)
Epoch 012/100 | Loss 0.0029 | Win/lose count 6.5/3.0 (3.5)
Epoch 013/100 | Loss 0.0026 | Win/lose count 6.5/5.0 (1.5)
Epoch 014/100 | Loss 0.0042 | Win/lose count 0.5/2.0 (-1.5)
Epoch 015/100 | Loss 0.0031 | Win/lose count 3.5/2.0 (1.5)
Epoch 016/100 | Loss 0.0057 | Win/lose count 5.5/2.0 (3.5

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [28]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

/Users/mac/Downloads/Python/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(input_shape=(5, 5, 2), activation="relu", kernel_size=(5, 5), filters=32, padding="same")`
  from ipykernel import kernelapp as app
/Users/mac/Downloads/Python/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same")`
/Users/mac/Downloads/Python/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(activation="relu", kernel_size=(5, 5), filters=64, padding="same")`
/Users/mac/Downloads/Python/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:31: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), padding="same")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 5, 5, 128)         384       
_________________________________________________________________
flatten_9 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dense_17 (Dense)             (None, 4)                 12804     
Total params: 13,188
Trainable params: 13,188
Non-trainable params: 0
_________________________________________________________________
None
Test of the CNN
Win/lose count 19.5/4.0. Average score (15.5)
Win/lose count 11.0/2.0. Average score (12.25)
Win/lose count 13.0/2.0. Average score (11.833333333333334)
Win/lose count 16.0/3.0. Average score (12.125)
Win/lose count 15.0/8.0. Average score (11.1)
Win/lose count 22.5/4.0. Average score (12.333333333333334)
Win/lose count 9.5/3.0. Average score (11.5)
Win/lose count 2.5/1.0. Avera

KeyboardInterrupt: 

In [ ]:
HTML(display_videos('cnn_test10.mp4'))

In [ ]:
HTML(display_videos('fc_test10.mp4'))

__Compare:__   The CNN Algorithm performs better than FC agent. 

__Issue:__    But there is an issue that the rat first explores the graph but then it will be trapped in a small area where he has already explored before. So there is a large part of the cells the rat will not explore. 

__Different Behaviors:__   The higher the temperature is, the better performance will be given by CNN Algorithm. 


***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [33]:
def train_explore(agent,env,epoch,prefix=''):

    # Number of won games
    score = 0
    loss = 0
    # Incorporating a decreasing  ϵϵ -greedy exploration.    
    min_epsilon = agent.epsilon
    max_epsilon = 0.2
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False
        
        win = 0
        lose = 0
        
        # epsilon decay
        if max_epsilon - 0.005 > min_epsilon:
            max_epsilon = max_epsilon-0.005 
        else:
            max_epsilon = min_epsilon
       
        agent.set_epsilon(max_epsilon)
        
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)   
            
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state, action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        

    
    
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))


        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0
        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, 
                                 grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, 
                        interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action,train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1 
        self.position[:,0:2] = -1 
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        ## In Environment exploring:
        # You will have to change n_state to 3 because you will use one more layer!
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        
        game_over = self.t > self.max_time

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        return state, reward, game_over

    
    def reset(self):
        """This function resets the game and returns the initial state"""
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        
        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state



In [34]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

/Users/mac/Downloads/Python/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(input_shape=(5, 5, 3), activation="relu", kernel_size=(5, 5), filters=32, padding="same")`
  from ipykernel import kernelapp as app
/Users/mac/Downloads/Python/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same")`
/Users/mac/Downloads/Python/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(activation="relu", kernel_size=(5, 5), filters=64, padding="same")`
/Users/mac/Downloads/Python/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:31: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), padding="same")`


Epoch 000/100 | Loss 0.0172 | Win/lose count 4.0/25.60000000000008 (-21.60000000000008)
Epoch 001/100 | Loss 0.0156 | Win/lose count 10.5/27.30000000000009 (-16.80000000000009)
Epoch 002/100 | Loss 0.0033 | Win/lose count 10.0/19.30000000000001 (-9.300000000000011)
Epoch 003/100 | Loss 0.0147 | Win/lose count 9.0/17.899999999999995 (-8.899999999999995)
Epoch 004/100 | Loss 0.0137 | Win/lose count 7.5/18.600000000000005 (-11.100000000000005)
Epoch 005/100 | Loss 0.0095 | Win/lose count 7.5/23.200000000000014 (-15.700000000000014)
Epoch 006/100 | Loss 0.0094 | Win/lose count 17.0/17.099999999999987 (-0.09999999999998721)
Epoch 007/100 | Loss 0.0085 | Win/lose count 7.5/19.800000000000008 (-12.300000000000008)
Epoch 008/100 | Loss 0.0142 | Win/lose count 16.0/20.00000000000002 (-4.000000000000021)
Epoch 009/100 | Loss 0.0135 | Win/lose count 17.5/14.89999999999997 (2.60000000000003)
Epoch 010/100 | Loss 0.0155 | Win/lose count 12.0/18.299999999999994 (-6.299999999999994)
Epoch 011/100 | L

Epoch 093/100 | Loss 0.0293 | Win/lose count 26.0/16.89999999999999 (9.100000000000009)
Epoch 094/100 | Loss 0.0245 | Win/lose count 18.5/23.800000000000068 (-5.300000000000068)
Epoch 095/100 | Loss 0.0298 | Win/lose count 21.5/20.80000000000005 (0.6999999999999496)
Epoch 096/100 | Loss 0.0289 | Win/lose count 19.5/16.899999999999984 (2.6000000000000156)
Epoch 097/100 | Loss 0.0173 | Win/lose count 22.0/16.199999999999974 (5.800000000000026)
Epoch 098/100 | Loss 0.0275 | Win/lose count 20.0/14.599999999999978 (5.400000000000022)
Epoch 099/100 | Loss 0.0365 | Win/lose count 22.0/22.700000000000017 (-0.700000000000017)


In [35]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 24.5/10.0. Average score (14.5)
Win/lose count 18.5/10.0. Average score (11.5)
Win/lose count 26.5/4.0. Average score (15.166666666666666)
Win/lose count 24.5/5.0. Average score (16.25)
Win/lose count 15.5/5.0. Average score (15.1)
Win/lose count 17.5/3.0. Average score (15.0)
Win/lose count 24.0/5.0. Average score (15.571428571428571)
Win/lose count 21.0/6.0. Average score (15.5)
Win/lose count 23.0/5.0. Average score (15.777777777777779)
Win/lose count 18.0/6.0. Average score (15.4)
Final score: 15.4


FileNotFoundError: [Errno 2] No such file or directory: 'cnn_test_explore10.mp4'

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***